### Dataset "Waste Type"

A quick script to clean and format the dataset.  
The information was gathered from the [Dolomiti Ambiente website](https://dolomitiambiente.it/it/), specifically from their [Reciclabolario](https://dolomitiambiente.it/it/trento/domestica/raccolta-differenziata/riciclabolario).


In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
df = pd.read_csv("waste_type_original.csv", header=None)

In [79]:
df.head()

,0
0,ABBIGLIAMENTO - CR
1,ACCENDINO - RESIDUO
2,ACCUMULATORI PER AUTO - CR
3,ACETONE E ACQUARAGIA - CR
4,ACETONE E ACQUARAGIA (contenitore vuoto e risc...


In [ ]:
#Adding new columns
df = df.rename(columns={0: 'waste_type'})
df["waste_notes"]=0
df["disposal_type_1"]=0
df["notes_1"]=0
df["disposal_type_2"]=0
df["notes_2"]=0

In [23]:
df.columns

Index(['waste_type', 'disposal_type_1', 'notes_1', 'disposal_type_2',
       'notes_2'],
      dtype='object')

In [32]:
df.head()

,waste_type,waste_notes,disposal_type_1,notes_1,disposal_type_2,notes_2
0,ABBIGLIAMENTO - CR,0,0,0,0,0
1,ACCENDINO - RESIDUO,0,0,0,0,0
2,ACCUMULATORI PER AUTO - CR,0,0,0,0,0
3,ACETONE E ACQUARAGIA - CR,0,0,0,0,0
4,ACETONE E ACQUARAGIA (contenitore vuoto e risc...,0,0,0,0,0


In [ ]:
#Splitting the 'waste_type' column at the first occurrence of "-" and expanding into two columns ("waste_type" and "dispoal_type_1")
df[['waste_type', 'disposal_type_1']] = df['waste_type'].str.split(' - ', n=1, expand=True)
df.head()

,waste_type,waste_notes,disposal_type_1,notes_1,disposal_type_2,notes_2
0,ABBIGLIAMENTO,0,CR,0,0,0
1,ACCENDINO,0,RESIDUO,0,0,0
2,ACCUMULATORI PER AUTO,0,CR,0,0,0
3,ACETONE E ACQUARAGIA,0,CR,0,0,0
4,ACETONE E ACQUARAGIA (contenitore vuoto e risc...,0,IMBALLAGGI LEGGERI,0,0,0


In [ ]:
#Splitting 'disposal_type_1' at the first occurrence of ';' and expanding into two columns ("dispoal_type_1" and "disposal_type_2")
df[['disposal_type_1', 'disposal_type_2']] = df['disposal_type_1'].str.split(';', n=1, expand=True)
df['disposal_type_2'] = df['disposal_type_2'].str.strip()
df.head(10)

,waste_type,waste_notes,disposal_type_1,notes_1,disposal_type_2,notes_2
0,ABBIGLIAMENTO,0,CR,0,None,0
1,ACCENDINO,0,RESIDUO,0,None,0
2,ACCUMULATORI PER AUTO,0,CR,0,None,0
3,ACETONE E ACQUARAGIA,0,CR,0,None,0
4,ACETONE E ACQUARAGIA (contenitore vuoto e risc...,0,IMBALLAGGI LEGGERI,0,None,0
5,ACIDI,0,CR,0,None,0
6,ADDOBBI NATALIZI,0,RESIDUO,0,None,0
7,AGHI DA SIRINGA,0,FARMACIA,0,None,0
8,ALBERO DI NATALE IN MATERIALE SINTETICO,0,RESIDUO,0,CR (se grande),0
9,ALBERO DI NATALE NATURALE,0,CR,0,None,0


In [ ]:
#Extracting text within parentheses and moving it to the corresponding notes columns
df['waste_notes'] = df['waste_type'].str.extract(r'\((.*?)\)')[0]
df['notes_1'] = df['disposal_type_1'].str.extract(r'\((.*?)\)')[0]
df['notes_2'] = df['disposal_type_2'].str.extract(r'\((.*?)\)')[0]

#Removing the text within parentheses from the original columns
df['waste_type'] = df['waste_type'].str.replace(r'\s*\(.*?\)', '', regex=True)
df['disposal_type_1'] = df['disposal_type_1'].str.replace(r'\s*\(.*?\)', '', regex=True)
df['disposal_type_2'] = df['disposal_type_2'].str.replace(r'\s*\(.*?\)', '', regex=True)

#Replacing None with NaN
df = df.replace({None: np.nan})

#Adding a new column 'index' with the row numbers starting from 1
df.insert(0, 'index', range(1, len(df) + 1))

In [99]:
df.head(20)

,index,waste_type,waste_notes,disposal_type_1,notes_1,disposal_type_2,notes_2
0,1,ABBIGLIAMENTO,NaN,CR,NaN,NaN,NaN
1,2,ACCENDINO,NaN,RESIDUO,NaN,NaN,NaN
2,3,ACCUMULATORI PER AUTO,NaN,CR,NaN,NaN,NaN
3,4,ACETONE E ACQUARAGIA,NaN,CR,NaN,NaN,NaN
4,5,ACETONE E ACQUARAGIA,contenitore vuoto e risciacquato,IMBALLAGGI LEGGERI,NaN,NaN,NaN
5,6,ACIDI,NaN,CR,NaN,NaN,NaN
6,7,ADDOBBI NATALIZI,NaN,RESIDUO,NaN,NaN,NaN
7,8,AGHI DA SIRINGA,NaN,FARMACIA,NaN,NaN,NaN
8,9,ALBERO DI NATALE IN MATERIALE SINTETICO,NaN,RESIDUO,NaN,CR,se grande
9,10,ALBERO DI NATALE NATURALE,NaN,CR,NaN,NaN,NaN


In [100]:
df.waste_type.unique()

array(['ABBIGLIAMENTO', 'ACCENDINO', 'ACCUMULATORI PER AUTO',
       'ACETONE E ACQUARAGIA', 'ACIDI', 'ADDOBBI NATALIZI',
       'AGHI DA SIRINGA', 'ALBERO DI NATALE IN MATERIALE SINTETICO',
       'ALBERO DI NATALE NATURALE', 'ALCOOL', 'ALLUMINIO', 'AMIANTO',
       'AMMONIACA', 'ANTIPARASSITARI', 'ANTITARME',
       'APPARECCHIATURE ELETTRICHE ED ELETTRONICHE', 'APPENDINO',
       'ARMADIO', 'ASCIUGACAPELLI', 'ASPIRAPOLVERE', 'ASSE DA STIRO',
       'ASSORBENTI', 'ASTUCCIO', 'ATTACCAPANNI',
       'BARATTOLO IN METALLO LATTA/PLASTICA', 'BARATTOLO IN VETRO',
       'BASTONCINI PER CARAMELLE “LOLLIPOP”', 'BATTERIA D’AUTOMOBILE',
       'BATTERIE ESAUSTE', 'BATUFFOLO DI COTONE', 'BIANCHERIA INTIMA',
       'BICCHIERE BIODEGRADABILE IN MATER BI', 'BICCHIERE IN PLASTICA',
       'BICCHIERE IN VETRO', 'BICICLETTA', 'BIGIOTTERIA',
       'BIGLIA IN VETRO', 'BLISTER', 'BOMBOLE DEL GAS',
       'BOMBOLETTE SPRAY', 'BORSE / BORSETTE',
       'BORSE PER LA SPESA IN MATER BI',
       'BORSE PER 

In [121]:
df.disposal_type_1.unique()

array(['CR', 'RESIDUO', 'IMBALLAGGI LEGGERI', 'FARMACIA',
       'DITTA SPECIALIZZATA', 'VETRO', 'ORGANICO',
       'RIVENDITORE AUTORIZZATO', 'CARTA E CARTONE', 'FARMACIA '],
      dtype=object)

In [122]:
df.disposal_type_2.unique()

array([nan, 'CR', 'DITTA SPECIALIZZATA', 'FARMACIA', 'IMBALLAGGI LEGGERI',
       'RIVENDITORI AUTORIZZATI'], dtype=object)

In [123]:
df.notes_1.unique()

array([nan, 'se grandi quantità', 'grandi dimensioni',
       'max 30 Kg/giorno e 5 volte/anno', 'fino a 5 lt', 'se pieni',
       'se grande'], dtype=object)

In [125]:
df.notes_2.unique()

array([nan, 'se grande', 'se grandi quantità', 'se grandi',
       'se grandi dimensioni', 'se in grandi quantità',
       'se in buono stato', 'in modesta quantità',
       'max 30 Kg/giorno e 5 volte/anno', 'max 60x40 cm',
       'se grandi quantità, più di 12 pezzi', 'se elettrica',
       'oltre 5 lt', 'se pieni'], dtype=object)

In [114]:
df.waste_notes.unique()

array([nan, 'contenitore vuoto e risciacquato', 'contenitore',
       'contenitore vuoto e pulito', 'in legno',
       'in plastica e in metallo', 'in ferro, in legno e in plastica',
       'conformi norma UNI EN 13432:2002', 'plastica e alluminio',
       'da campeggio',
       'vuote e prive di simboli di pericolo e piccole quantità',
       'con simbolo T e/o F', 'in stoffa', 'sintetiche',
       'salvo diversa indicazione sulla confezione',
       'in modesta quantità', 'gomma dell’acqua',
       'in plastica e/o alluminio priva di contenuto',
       'in plastica e/o alluminio sporche', 'da legna', 'da altre fonti',
       'non pericolosa e in piccole quantità',
       'es. estetiste, studio medico, …', 'in alluminio, pulita',
       'lavamani', 'chimiche', 'completi di connettori',
       'grandi dimensioni maggiore 60x40cm', 'di legna purché spenta',
       'in plastica, stoffa o cuoio in buono stato',
       'in plastica, stoffa o cuoio inutilizzabile',
       'porta uova, scato

In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('waste_type.csv', index=False)